# Binary decision tree classifier 
### from scratch using turicreate

In [5]:
#working notebook theme
import jupyterthemes as jt
jt.stylefx.set_nb_theme('gruvboxl')

In this notebook we implement a binary decision tree classifier following U. of Washington's course on ML Classification.

In [2]:
import turicreate

###  Load the lending club dataset

We will use a dataset from [LendingClub](https://www.lendingclub.com/).

In [3]:
loans = turicreate.SFrame('lending-club-data.sframe/')

For consistency we reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan.

In [4]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.remove_column('bad_loans')

### We visualize some of the columns

In [100]:
loans.column_names()

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'is_inc_v',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'not_compliant',
 'status',
 'inactive_loans',
 'emp_length_num',
 'grade_num',
 'sub_grade_num',
 'delinq_2yrs_zero',
 'pub_rec_zero',
 'coll

In [98]:
turicreate.visualization.set_target('auto')
print(loans['grade'].show())
loans['home_ownership'].show()

Materializing SArray

None


Materializing SArray

We will just be using 4 categorical features: 

1. grade of the loan 
2. the length of the loan term
3. the home ownership status: own, mortgage, rent
4. number of years of employment.

Since we are building a binary decision tree, we will convert these categorical features to a binary representation in a subsequent section using 1-hot encoding.

In [5]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

Let's explore what the dataset looks like.

In [104]:
loans

grade,term,home_ownership,emp_length,safe_loans
B,36 months,RENT,10+ years,1
C,60 months,RENT,< 1 year,-1
C,36 months,RENT,10+ years,1
C,36 months,RENT,10+ years,1
A,36 months,RENT,3 years,1
E,36 months,RENT,9 years,1
F,60 months,OWN,4 years,-1
B,60 months,RENT,< 1 year,-1
C,60 months,OWN,5 years,1
B,36 months,OWN,10+ years,1


## Subsample dataset to obtain balanced classes

In [108]:
loans['safe_loans'].show(title='class distribution before resampling')

Materializing SArray

We will undersample the larger class (safe loans) in order to balance out our dataset. This is the simplest approach to data imbalance.

In [6]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(percentage, seed = 1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print("Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data)))
print("Percentage of risky loans                :", len(risky_loans) / float(len(loans_data)))
print("Total number of loans in our new dataset :", len(loans_data))

('Percentage of safe loans                 :', 0.5022361744216048)
('Percentage of risky loans                :', 0.4977638255783951)
('Total number of loans in our new dataset :', 46508)


**Note:** There are many approaches for dealing with imbalanced data, including some where we modify the learning algorithm as reviewed in "Learning from Imbalanced Data" by Haibo He and Edwardo A. Garcia, *IEEE Transactions on Knowledge and Data Engineering* **21**(9) (June 26, 2009), p. 1263–1284. 

## Transform categorical data into binary features

We will implement **binary decision trees** (decision trees for binary features, a specific case of categorical variables taking on two values, e.g., true/false). Since all of our features are currently categorical features, we want to turn them into binary features. 

For instance, the **home_ownership** feature represents the home ownership status of the loanee, which is either `own`, `mortgage` or `rent`. For example, if a data point has the feature 
```
   {'home_ownership': 'RENT'}
```
we want to turn this into three features: 
```
 { 
   'home_ownership = OWN'      : 0, 
   'home_ownership = MORTGAGE' : 0, 
   'home_ownership = RENT'     : 1
 }
```

We will use the ``unpack`` method which corresponds to ``get_dummies`` in Pandas.

In [7]:
loans_data = risky_loans.append(safe_loans)
for feature in features:
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})    
    loans_data_unpacked = loans_data_one_hot_encoded.unpack(column_name_prefix=feature)
    
    # Change None's to 0's
    for column in loans_data_unpacked.column_names():
        loans_data_unpacked[column] = loans_data_unpacked[column].fillna(0)

    loans_data = loans_data.remove_column(feature)
    loans_data = loans_data.add_columns(loans_data_unpacked)

Let's see what the feature columns look like now:

In [8]:
features = loans_data.column_names()
features.remove('safe_loans')  # Remove the response variable
features

['grade.A',
 'grade.B',
 'grade.C',
 'grade.D',
 'grade.E',
 'grade.F',
 'grade.G',
 'term. 36 months',
 'term. 60 months',
 'home_ownership.MORTGAGE',
 'home_ownership.OTHER',
 'home_ownership.OWN',
 'home_ownership.RENT',
 'emp_length.1 year',
 'emp_length.10+ years',
 'emp_length.2 years',
 'emp_length.3 years',
 'emp_length.4 years',
 'emp_length.5 years',
 'emp_length.6 years',
 'emp_length.7 years',
 'emp_length.8 years',
 'emp_length.9 years',
 'emp_length.< 1 year',
 'emp_length.n/a']

In [26]:
print("Number of features (after binarizing categorical variables) = %s" % len(features))

Number of features (after binarizing categorical variables) = 25


Let's explore what one of these columns looks like:

In [27]:
loans_data['grade.A']

dtype: int
Rows: 46508
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, ... ]

This column is set to 1 if the loan grade is A and 0 otherwise.

## Train-test split

We split the data into a train test split with 80% of the data in the training set and 20%.

In [34]:
train_data, test_data = loans_data.random_split(.8, seed=1)

# Decision tree implementation

In this section, we will implement binary decision trees from scratch. 

### Function to count number of mistakes while predicting majority class

Note that prediction at an intermediate node works by predicting the **majority class** for all data points that belong to this node.

First we will write a function that calculates the number of **missclassified examples** when predicting the **majority class**. This will be used to help determine which feature is the best to split on at a given node of the tree. Note that we only need the label (y values) of the data points in the node. 

In [37]:
def intermediate_node_num_mistakes(labels_in_node):
    #this functions counts the number of mistakes made
    #at a single node. The node will assign the class
    #of the majority to its prediction, so the number of mistakes
    #is the number of elements in the minority class
    
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    # Count the number of 1's (safe loans)
    number_pos = len(labels_in_node[labels_in_node == 1])
    
    # Count the number of -1's (risky loans)
    number_neg = len(labels_in_node[labels_in_node == -1])
    
    #return cardinality of the minority class
    if number_pos >= number_neg:
        number_err = number_neg
    else:
        number_err = number_pos
    
    return number_err

### Function to pick best feature to split on

The function will loop through the list of possible features, and consider splitting on each of them. It will calculate the classification error of each split and return the feature that had the smallest classification error when split on.


$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$


In [9]:
def best_splitting_feature(data, features, target):
    # data = SFrame of data with features and labels
    # features = list of strings of column names to consider for splits
    # target = label column (string)
    # The function returns the feature name associated with smallest misclassification error
    
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # We do split the data into'left' and 'right' subsets.
        # Recall we are working with binary attributes X in {0,1}
        left_split = data[data[feature] == 0]
        right_split =  data[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        left_mistakes = intermediate_node_num_mistakes(left_split[target])            

        # Calculate the number of misclassified examples in the right split.
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        error = (left_mistakes + right_mistakes)/num_data_points
        
        #keep smallest error and its associated feature
        if error < best_error:
            best_error = error
            best_feature = feature
        
    
    return best_feature # Return the best feature we found

### Building the tree

Each node in the decision tree is represented as a dictionary which contains the following keys and possible values:

    { 
       'is_leaf'            : True/False.
       'prediction'         : Prediction at the leaf node. In this implementation (+/- 1)
       'left'               : (dictionary corresponding to the left tree).
       'right'              : (dictionary corresponding to the right tree).
       'splitting_feature'  : The feature that this node splits on.
    }
    

First, we will write a function that creates a leaf node given a set of target values. 

In [11]:
def create_leaf(target_values):
    #given the target values make a leaf node with majority vote prediction
    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True     }  # is_leaf = True makes this node a leaf
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] =  +1        
    else:
        leaf['prediction'] = -1       
        
    # Return the leaf node        
    return leaf 

We have provided a function that learns the decision tree recursively and implements 3 stopping conditions:
1. **Stopping condition 1:** All data points in a node are from the same class.
2. **Stopping condition 2:** No more features to split on.
3. **Stopping condition 3:** We reach max tree depth.

In [102]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10, verbose = False):
    #function to learn a decision tree recursively.
    #Recursion stops if one the three conditions is met:
    # 1. all data points are of the same class,
    # 2. no more features so split on,
    # 3. or we reach the specified max_depth
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    
    if verbose:
        print("--------------------------------------------------------------------")
        print("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))


    # Stopping condition 1
    # Check if there are mistakes at current node.
    if  intermediate_node_num_mistakes(target_values) == 0:  
        if verbose:
            print("Stopping condition 1 reached.")
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if remaining_features == list(best_splitting_feature(data, features, target)):  
        if verbose:
            print("Stopping condition 2 reached.")
        return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth >= max_depth :  
        if verbose:
            print("Reached maximum depth. Stopping for now.")
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # Find the best splitting feature from remaining features
    splitting_feature = best_splitting_feature(data, features, target)

    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    # After split, we remove the used feature from the remaining_features
    remaining_features.remove(splitting_feature)
    if verbose:
        print("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        if verbose:
            print("Creating leaf node.")
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        if verbose:
            print("Creating leaf node.")
        return create_leaf(right_split['target'])

        
    # Recurse on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)        
    right_tree = decision_tree_create(right_split, remaining_features, target, current_depth + 1, max_depth)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

Here is a recursive function to count the nodes in the tree:

In [72]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

### Build the tree

We will train a tree model on the train_data. We limit the depth to 6 to make sure the algorithm doesn't run for too long. 

In [103]:
decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, verbose=False)

In [105]:
decision_tree

{'is_leaf': False,
 'left': {'is_leaf': False,
  'left': {'is_leaf': False,
   'left': {'is_leaf': False,
    'left': {'is_leaf': False,
     'left': {'is_leaf': False,
      'left': {'is_leaf': True,
       'left': None,
       'prediction': -1,
       'right': None,
       'splitting_feature': None},
      'prediction': None,
      'right': {'is_leaf': True,
       'left': None,
       'prediction': -1,
       'right': None,
       'splitting_feature': None},
      'splitting_feature': 'grade.E'},
     'prediction': None,
     'right': {'is_leaf': True,
      'left': None,
      'prediction': -1,
      'right': None,
      'splitting_feature': None},
     'splitting_feature': 'grade.D'},
    'prediction': None,
    'right': {'is_leaf': True,
     'left': None,
     'prediction': -1,
     'right': None,
     'splitting_feature': None},
    'splitting_feature': 'grade.C'},
   'prediction': None,
   'right': {'is_leaf': False,
    'left': {'is_leaf': True,
     'left': None,
     'predi

### Making predictions with a decision tree

We can make predictions from the decision tree with a simple recursive function. Below, we call this function `classify`, which takes in a learned `tree` and a test point `x` to classify.  We include an option `annotate` that describes the prediction path when set to `True`.

In [48]:
def classify(tree, x, annotate = False): 
    #function to make predictions using a learned tree
    # x = observation
    
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'],x,annotate)

Now, let's consider the first example of the test set and see what `my_decision_tree` model predicts for this data point.

In [49]:
test_data[0]

{'emp_length.1 year': 0,
 'emp_length.10+ years': 0,
 'emp_length.2 years': 1,
 'emp_length.3 years': 0,
 'emp_length.4 years': 0,
 'emp_length.5 years': 0,
 'emp_length.6 years': 0,
 'emp_length.7 years': 0,
 'emp_length.8 years': 0,
 'emp_length.9 years': 0,
 'emp_length.< 1 year': 0,
 'emp_length.n/a': 0,
 'grade.A': 0,
 'grade.B': 0,
 'grade.C': 0,
 'grade.D': 1,
 'grade.E': 0,
 'grade.F': 0,
 'grade.G': 0,
 'home_ownership.MORTGAGE': 0,
 'home_ownership.OTHER': 0,
 'home_ownership.OWN': 0,
 'home_ownership.RENT': 1,
 'safe_loans': -1,
 'term. 36 months': 0,
 'term. 60 months': 1}

In [50]:
print('Predicted class: %s ' % classify(decision_tree, test_data[0]))

Predicted class: -1 


Let's add some annotations to our prediction to see what the prediction path was that lead to this predicted class:

In [52]:
classify(decision_tree, test_data[0], annotate=True)

Split on term. 36 months = 0
Split on grade.A = 0
Split on grade.B = 0
Split on grade.C = 0
Split on grade.D = 1
At leaf, predicting -1


-1

### Evaluating the decision tree

We write a function to evaluate a decision tree by computing the classification error of the tree on the given dataset.


$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$


In [108]:
def evaluate_classification_error(tree, data, target):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x))
    
    num_mistakes = float(len(data[data[target] != prediction]))
    clsf_error = num_mistakes/len(data)
    
    
    return clsf_error
    

In [70]:
evaluate_classification_error(decision_tree, test_data, target)

0.3837785437311504

### Printing out a decision stump

In [71]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print("(leaf, label: %s)" % tree['prediction'])
        return None
    split_feature, split_value = split_name.split('.')
    print('                       %s' % name)
    print('         |---------------|----------------|')
    print('         |                                |')
    print('         |                                |')
    print('         |                                |')
    print('  [{0} == 0]               [{0} == 1]    '.format(split_name))
    print('         |                                |')
    print('         |                                |')
    print('         |                                |')
    print('    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree')))

In [73]:
print_stump(decision_tree)

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term. 36 months == 0]               [term. 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


### Exploring the intermediate left subtree

The tree is a recursive dictionary, so we do have access to all the nodes! We can use
* `my_decision_tree['left']` to go left
* `my_decision_tree['right']` to go right

In [76]:
print_stump(decision_tree['left'], decision_tree['splitting_feature'])

                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.A == 0]               [grade.A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


### Exploring the left subtree of the left subtree


In [77]:
print_stump(decision_tree['left']['left'], decision_tree['left']['splitting_feature'])

                       grade.A
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.B == 0]               [grade.B == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [79]:
print_stump(decision_tree['right'], decision_tree['splitting_feature'])

                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.D == 0]               [grade.D == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: -1)


In [87]:
print_stump(decision_tree['right'], decision_tree['right']['splitting_feature'])

                       grade.D
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.D == 0]               [grade.D == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: -1)
